In [1]:
import numpy as np
import re
import pronouncing
import random
import numpy as np
import markovify
import os
import keras
from keras.models import Sequential
from keras.layers import LSTM 
from keras.layers.core import Dense
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

Using TensorFlow backend.


In [2]:
artist_file = 'eminem_removed.txt'
data = []
with open(artist_file) as f:
    for line in f.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        data.append(line)
        #print(line)
print(data)

# Separate each line of the input txt
def split_lyrics_file(text_file):
	text = open(text_file, encoding='utf-8').read()
	text = text.split("\n")
	while "" in text:
		text.remove("")
	return text

['Look I was gonna go easy on you and not to hurt your feelings', 'But Im only going to get this one chance', 'Somethings wrong I can feel it Six minutes Slim Shady youre on', 'Just a feeling Ive got like somethings about to happen but I dont know what', 'If that means what I think it means were in trouble big trouble', 'And if he is as bananas as you say Im not taking any chances', 'You were just what the doctor ordered Im beginning to feel like a Rap God Rap God', 'All my people from the front to the back nod back nod', 'Now who thinks their arms are long enough to slap box slap box', 'They said I rap like a robot so call me Rapbot But for me to rap like a computer must be in my genes', 'I got a laptop in my back pocket', 'My penll go off when I halfcock it', 'Got a fat knot from that rap profit', 'Made a living and a killing off it', 'Ever since Bill Clinton was still in office', 'With Monica Lewinsky feeling on his nutsack', 'Im an MC still as honest', 'But as rude and indecent as 

In [3]:
def create_network(depth):
        model = Sequential()
        #input_shape(2,2) -> x_data = [[syllables_num,rhyme_num],[syllables_num,rhyme_num]]
        #第一个list里是第一句歌词的音节和押韵，第二个list是第二句歌词的音节和押韵
        model.add(LSTM(4, input_shape=(2, 2), return_sequences=True)) # 返回维度为 4 的向量序列, input_shape=(timesteps, data_dim)
        for i in range(depth):
                model.add(LSTM(8, return_sequences=True))# 返回维度为 8 的向量序列
        model.add(LSTM(2, return_sequences=True))# 返回维度为 2 的向量序列
        model.summary()
        model.compile(optimizer='rmsprop', loss='mse')
        if artist + ".rap" in os.listdir(".") and train_mode == False:
                model.load_weights(str(artist + ".rap"))
                print("loading saved network: " + str(artist) + ".rap") 
        return model

In [4]:
#build new sentences (based off of word1=>word2 probabilities)
def markov(text_file):
    # Markov Chain
    read = open(text_file, "r", encoding='utf-8').read()
    text_model = markovify.NewlineText(read)
    return text_model

In [5]:
#Determine number of syllables in line
def syllables(line):
    count = 0
    for word in line.split(" "):#句子中每个单词
        vowels = 'aeiouy'
        word = word.lower().strip("!@#$%^&*()_+-={}[];:,.<>/?-")
        if len(word) == 0:
            continue
        if word[0] in vowels: #如果单词开头是元音字母，+1
            count +=1
            
        for index in range(1,len(word)):#如果前一个字母是元音后一个不是，+1
            if word[index] in vowels and word[index-1] not in vowels:
                count +=1
        if word.endswith('e'):#如果最后单词最后一个字母是e，-1
            count -= 1
        if word.endswith('le'):#如果单词最后结尾是le,+1
            count+=1
        if count == 0: #防止算法出错
            count +=1
    return count / maxsyllables


In [6]:
#Make index of words that rhyme with the word 返回一个包含所有韵脚的list
def rhymeindex(lyrics):
        if str(artist) + ".rhymes" in os.listdir(".") and train_mode == False:#如果文档已经创建并且没有开始训练，则读取文档并返回一个包含韵脚的list
                print ("loading saved rhymes from " + str(artist) + ".rhymes")
                return open(str(artist) + ".rhymes", "r",encoding='utf-8').read().split("\n")
        else:
                rhyme_master_list = []
                print ("Building list of rhymes:")
                for i in lyrics:
                        word = re.sub(r"\W+", '', i.split(" ")[-1]).lower()#每一句歌词最后一个单词
                        rhymeslist = pronouncing.rhymes(word)#每个单词对应的押韵单词
                        #ex:aughter->rhyme ['boughter', 'bowater', 'bowater', 'lawter', 'sautter', 'slaughter', 'straughter', 'vawter', 'water']
                        rhymeslistends = [] #每个单词最后两个字母 ex：teacher,daughter->er    
                        for i in rhymeslist:
                                rhymeslistends.append(i[-2:])
                        #保存出现最多的韵脚字母 ex：er
                        try:
                                rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count) 
                        except Exception:
                                rhymescheme = word[-2:]
                        rhyme_master_list.append(rhymescheme)
                rhyme_master_list = list(set(rhyme_master_list))#rhyme_master_list:删除重复的韵脚，只留下一个
                reverselist = [x[::-1] for x in rhyme_master_list]
                reverselist = sorted(reverselist)
                rhymelist = [x[::-1] for x in reverselist] #将韵脚按顺序排序
                print("List of Sorted 2-Letter Rhyme Ends:")
                print(rhymelist)
                #将韵脚按顺序写入
                f = open(str(artist) + ".rhymes", "w", encoding='utf-8')
                f.write("\n".join(rhymelist))
                f.close()
        return rhymelist
    

In [7]:
# Make index of rhymes that you use
def rhyme(line, rhyme_list):
    #-------------------
    '''
    这一节和rhymeindex(lyrics)是一样的'''
    word = re.sub(r"\W+", '', line.split(" ")[-1]).lower()#每一句歌词最后一个单词
    rhymeslist = pronouncing.rhymes(word)#每个单词对应的押韵单词
    rhymeslistends = []#每个单词最后两个字母 ex：teacher,daughter->er 
    for i in rhymeslist:
        rhymeslistends.append(i[-2:])
    #保存出现最多的韵脚字母 ex：er
    try:
        rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
    except Exception:
        rhymescheme = word[-2:]
    
   #---------------------



    #返回一个小数
    try:
        float_rhyme = rhyme_list.index(rhymescheme)#韵脚字母出现在押韵list的index
        float_rhyme = float_rhyme / float(len(rhyme_list))#len(rhyme_list)=192
        return float_rhyme
    except Exception:
        float_rhyme = None
        return float_rhyme
    

In [8]:
# Generate lyrics
#Computational time will increase dramatically when maxsyllables < 8 and when max_overlap_ratio < 0.5’‘’
def generate_lyrics(text_model, text_file):
    bars = []
    last_words = []
    lyriclength = len(open(text_file,encoding='utf-8').read().split("\n"))#和输入文件的长度相同
    count = 0
    markov_model = markov(text_file)#build new sentences (based off of word1=>word2 probabilities)

    while len(bars) < lyriclength / 9 and count < lyriclength * 2:
        bar = markov_model.make_sentence(max_overlap_ratio = .49, tries=100)
        if type(bar) != type(None) and syllables(bar) < 1:
            #获取bar中最后一个单词，如果带有符号则去掉符号
            def get_last_word(bar):
                last_word = bar.split(" ")[-1]
                if last_word[-1] in "!.?,":
                    last_word = last_word[:-1]
                return last_word
            last_word = get_last_word(bar)
            #如果新生成的bar不存在于 list中，则加入该bar
            if bar not in bars and last_words.count(last_word) < 3:
                bars.append(bar)
                last_words.append(last_word)
                count += 1
    return bars

In [9]:
#Build dataset
def build_dataset(lines, rhyme_list):
        dataset = []
        line_list = []
        for line in lines:
                line_list = [line, syllables(line), rhyme(line, rhyme_list)]
                dataset.append(line_list)
        x_data = []
        y_data = []
        for i in range(len(dataset) - 3):
                line1 = dataset[i    ][1:]
                line2 = dataset[i + 1][1:]
                line3 = dataset[i + 2][1:]
                line4 = dataset[i + 3][1:]
                #x=float(syllables_num,rhyme_num)
                #x_data = [array([[1.7, 0.71428571],[1.0, 0.23809524]])] shape:2x2 
                x = [line1[0], line1[1], line2[0], line2[1]]
                x = np.array(x)
                x = x.reshape(2,2)
                x_data.append(x)
                y = [line3[0], line3[1], line4[0], line4[1]]
                y = np.array(y)
                y = y.reshape(2,2)
                y_data.append(y)
        x_data = np.array(x_data)
        y_data = np.array(y_data)
        return x_data, y_data

In [10]:
# Compose verse
def compose_rap(lines, rhyme_list, lyrics_file, model):
    rap_vectors = []
    human_lyrics = split_lyrics_file(lyrics_file)
    #随机定位歌词位置并开始学习
    initial_index = random.choice(range(len(human_lyrics) - 1))
    initial_lines = human_lyrics[initial_index:initial_index + 2]
    starting_input = []
    for line in initial_lines:
        starting_input.append([syllables(line), rhyme(line, rhyme_list)])
    #输入测试数据,输出预测结果
    starting_vectors = model.predict(np.array([starting_input]).flatten().reshape(1, 2, 2))
    rap_vectors.append(starting_vectors)
    for i in range(100):
        rap_vectors.append(model.predict(np.array([rap_vectors[-1]]).flatten().reshape(1, 2, 2)))
    #预测结果：3-D: 1x2x2; 输出前后两句的syllables和rhyme的预测值
    return rap_vectors

def vectors_into_song(vectors, generated_lyrics, rhyme_list):
    print ("\n\n")	
    print ("Writing verse:")
    print ("\n\n")
    
    #计算歌词的penalty：前后两句的最后一个歌词是否相同，相同则penalty+0.2
    def last_word_compare(rap, line2):
        penalty = 0 
        for line1 in rap:
            word1 = line1.split(" ")[-1]
            word2 = line2.split(" ")[-1]
            while word1[-1] in "?!,. ":
                word1 = word1[:-1]
            while word2[-1] in "?!,. ":
                word2 = word2[:-1]
            if word1 == word2:
                penalty += 0.2
        return penalty
    #计算每一句歌词的分数
    #score = 1 - abs(desired_syllables - syllables) + abs(desired_rhyme - rhyme) - penalty
    def calculate_score(vector_half, syllables, rhyme, penalty):
        desired_syllables = vector_half[0]#syllables预测值
        desired_rhyme = vector_half[1]#rhyme预测值
        desired_syllables = desired_syllables * maxsyllables
        desired_rhyme = desired_rhyme * len(rhyme_list)
        fds = float(desired_syllables)
        fs = float(syllables)
        fdr = float(desired_rhyme)
        fr = float(rhyme)
        score = 1.0 - abs(fds - fs) + abs(fdr - fr) - penalty
        return score
    dataset = []
    for line in generated_lyrics:
        line_list = [line, syllables(line), rhyme(line, rhyme_list)]
        dataset.append(line_list)
    rap = []
    vector_halves = []
    for vector in vectors:
        vector_halves.append(list(vector[0][0])) 
        vector_halves.append(list(vector[0][1]))
    for vector in vector_halves:
        scorelist = []
        for item in dataset:
            line = item[0]
            if len(rap) != 0:
                penalty = last_word_compare(rap, line)
            else:
                penalty = 0
            total_score = calculate_score(vector, item[1], item[2], penalty)
            score_entry = [line, total_score]
            scorelist.append(score_entry)
        fixed_score_list = [0]
        for score in scorelist:
            fixed_score_list.append(float(score[1]))
        max_score = max(fixed_score_list)
        #如果预测歌曲的得分是最大得分，则选取这句歌词
        #score = 1 - abs(desired_syllables - syllables) + abs(desired_rhyme - rhyme) - penalty
        #如果是重复的歌词则删除
        for item in scorelist:
            if item[1] == max_score:
                rap.append(item[0])
                print (str(item[0]))
                for i in dataset:
                    if item[0] == i[0]:
                        dataset.remove(i)
                        break
                break     
    return rap

In [11]:
#Traning function
def train(x_data, y_data, model):
        model.fit(np.array(x_data), np.array(y_data),
                          batch_size=2,
                          epochs=5,
                          verbose=1)
        model.save_weights(artist + ".rap")

In [12]:
#Train and run the model
def main(depth, train_mode):
        model = create_network(depth)
        text_model = markov(text_file)
        if train_mode == True:
                bars = split_lyrics_file(text_file)
        if train_mode == False:
                bars = generate_lyrics(text_model, text_file)
        rhyme_list = rhymeindex(bars)
        if train_mode == True:
                x_data, y_data = build_dataset(bars, rhyme_list)
                train(x_data, y_data, model)
        if train_mode == False:
                vectors = compose_rap(bars, rhyme_list, text_file, model)
                rap = vectors_into_song(vectors, bars, rhyme_list)
                f = open(rap_file, "w", encoding='utf-8')
                for bar in rap:
                        f.write(bar)
                        f.write("\n")

In [ ]:
depth = 8
artist = "artist"
rap_file = "temporary_poem.txt"

maxsyllables = 16
text_file = "eminem_removed.txt"
train_mode = True        
main(depth, train_mode)
train_mode = False
main(depth, train_mode)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2, 4)              112       
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 8)              416       
_________________________________________________________________
lstm_3 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_4 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_5 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_6 (LSTM)                (None, 2, 8)              544       
_________________________________________________________________
lstm_7 (LSTM)                (None, 2, 8)             